In [24]:
from time import sleep
from spi_helper import *
from litex import RemoteClient
from ad9174 import Ad9174Settings, Ad9174Init

def getId(r):
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s

# generate a PRBS7 bit-pattern to compare against
a = 2
for i in range(255):
    nb = (((a >> 6) ^ (a >> 5)) & 1)
    a = ((a << 1) | nb)
prbs7_str = "{:b}".format(a)
prbs7_str_inv = "{:b}".format(a ^ (2**256 - 1))

def describePattern(bVal):
    ''' identifies some common test patterns '''
    desc = ''
    if bVal.find('00111110101100000101') >= 0:
        desc = ','
    elif prbs7_str.find(bVal) >= 0:
        desc = 'PRBS7'
    elif prbs7_str_inv.find(bVal) >= 0:
        desc = '/PRBS7'
    return desc

# Connec to FPGA

In [2]:
r = RemoteClient(csr_csv='../build/csr.csv', debug=False, port=1234)
r.open()
getId(r)

'AD9174 + VC707 test 2021-02-21 22:54:45'

# Setup AD9174

In [3]:
settings = Ad9174Settings(json_file='../build/csr.json')
print(settings)
ad = Ad9174Init(r, settings)
wr = ad.ad.wr
rr = ad.ad.rr

ad.hmc.init_hmc7044_int(N2=20)
ad.hmc.setup_channel( 2, 2)    # Clock to the AD9174
ad.hmc.setup_channel(12, 8)    # DSP clock to the FPGA
ad.hmc.setup_channel( 3, 800)  # SYSREF to AD9174
ad.hmc.setup_channel(13, 800)  # SYSREF to FPGA
ad.hmc.trigger_div_reset()

ad.init_ad9174(ADC_CLK_DIV=4, USE_PLL=True, M_DIV=2, N_DIV=2, OUT_DIV=2)

ad.fpga_print_clocks()

----------------
 JESD mode 20
----------------
INTERP_CH: 1  INTERP_MAIN: 1  DSP_CLK_DIV: 16
JESD204BSettings(): 5a 05 00 87 00 1f 00 0f 2f 23 80 00 00 e6 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   8        SCR:   1 
           F:   1          K:  32          M:   1          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   4 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 230 
   [ LINK_DW:  32     FR_CLK:   4 ]
AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5
DAC PLL locked: 1
DLL locked: 1
SPI_PAGEINDX: 0b01000001
CAL_STAT: 1
SERDES PLL locked: 1
MODE_NOT_IN_TABLE: 0
f_jesd = 307.197416 MHz  f_ref = 3.071974 MHz


# Reset the GTX phy

In [4]:
# r.regs.ctrl_reset.write(1)  # resets ALL clockdomains (HARSH!)
r.regs.control_control.write(0b01)  # resets PHYs and jesd core
print('status: {:03b}'.format(r.regs.control_status.read()))

# bit1: links_enable,  bit0: phys_reset
r.regs.control_control.write(0b00)

status: 000


In [5]:
# bit2: /jsync,  bit1: links_ready,  bit0: all phys initialized
print('status: {:03b}'.format(r.regs.control_status.read()))

status: 101


# GTX testpattern --> AD9174 snapshot

In [42]:
# Test pattern modes of the liteJESD phy
# 0 = normal operation
# 1 = prbs7
# 2 = prbs15
# 3 = prbs31
# 4 = static testpattern 
#     by default a square wave of freq. line_rate / 40
val = 1
for i in range(8):
    getattr(r.regs, 'phy{}_transmitter_config'.format(i)).write(val)

r.regs.phy0_transmitter_config.read()

1

If the pattern is stable on all lanes and does not change when running the below cell multiple times, the GTXes are locked.

`K28.5` = 0011111010 / 1100000101 

In [43]:
wr('PHY_PRBS_TEST_EN', 0xFF)  # Needed: clock to test module
wr('PHY_PRBS_TEST_CTRL', 0b01)  # rst

print('PHY_SNAPSHOT_DATA:')
for lane in range(8):
    wr('PHY_PRBS_TEST_CTRL', (lane << 4))    
    wr('PHY_DATA_SNAPSHOT_CTRL', 0x01)
    wr('PHY_DATA_SNAPSHOT_CTRL', 0x00)
    val = 0
    for i in range(5):
        val = (val << 8) | rr(0x323 - i)
    bVal = '{:040b}'.format(val)
    print('{0:}: 0x{1:010x}, 0b{2:}, {3:}'.format(lane, val, bVal, describePattern(bVal)))

PHY_SNAPSHOT_DATA:
0: 0x438936b7b1, 0b0100001110001001001101101011011110110001, PRBS7
1: 0x41851e459d, 0b0100000110000101000111100100010110011101, PRBS7
2: 0x8106147916, 0b1000000100000110000101000111100100010110, PRBS7
3: 0x49b5bd8d2e, 0b0100100110110101101111011000110100101110, PRBS7
4: 0x7d0e24dade, 0b0111110100001110001001001101101011011110, PRBS7
5: 0x830a3c8b3a, 0b1000001100001010001111001000101100111010, PRBS7
6: 0x8106147916, 0b1000000100000110000101000111100100010110, PRBS7
7: 0x2afe041851, 0b0010101011111110000001000001100001010001, PRBS7


# PRBS check

In [44]:
prbs = 0  # 0: PRBS7, 1: PRBS15, 2: PRBS31
wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)
wr('PHY_PRBS_TEST_EN', 0xFF)  # Enable test for all lanes
wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b01)  # rst
wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)
wr('PHY_PRBS_TEST_THRESHOLD_LOBITS', 0xFF)  # error threshold

wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b10)  # start test
sleep(0.01)
wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)  # stop test

print('PHY_PRBS_TEST_STATUS: {:08b} (high = pass)'.format(rr('PHY_PRBS_TEST_STATUS')))
for lane in range(8):
    wr('PHY_PRBS_TEST_CTRL', (lane << 4) | (prbs << 2) | 0b00)  # read-out lane X
    err_cnt = rr('PHY_PRBS_TEST_ERRCNT_HIBITS') << 16 | \
              rr('PHY_PRBS_TEST_ERRCNT_MIDBITS') << 8 | \
              rr('PHY_PRBS_TEST_ERRCNT_LOBITS')
    print('lane: {}, err_cnt: {:06x}'.format(lane, err_cnt))

PHY_PRBS_TEST_STATUS: 11111111 (high = pass)
lane: 0, err_cnt: 000000
lane: 1, err_cnt: 000000
lane: 2, err_cnt: 000000
lane: 3, err_cnt: 000000
lane: 4, err_cnt: 000000
lane: 5, err_cnt: 000000
lane: 6, err_cnt: 000000
lane: 7, err_cnt: 000000


# Setup for internal loopback PRBS check

In [32]:
wr(0x240, 0)
wr(0x241, 0)
# wr('CDR_BITINVERSE', 0x00)
wr('LBT_REG_CNTRL_0', 0xFF)  # enable loopback for lane X
halfrate = 0  # 1: enable halfrate mode (keep off!)
wr('LBT_REG_CNTRL_1', (halfrate << 1) | 1)
wr('LBT_REG_CNTRL_1', halfrate << 1)

0

In [34]:
rr(0x201)

0